In [2]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

In [3]:
def df_cluster(nits_clientes, distrito, fecha_inicial, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL_nits = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL =  f"SELECT A.VTANIT AS 'Nitcliente', B.NITNOM AS 'Nombre cliente', A.VTAPRDCOD AS 'Codigo producto',\
                        SUM(A.VTAUNDINV) AS 'Unidades' FROM V_VTA_VTAHEC A LEFT JOIN DIM_NITS B ON A.VTANIT = B.NITNIT\
                        WHERE A.VTANIT = '{nit}' AND A.VTAFCH BETWEEN '{fecha_inicial}' AND '{fecha_final}' AND A.DSTCOD = {distrito}\
                        GROUP BY A.VTANIT, B.NITNOM, A.VTAPRDCOD HAVING SUM(A.VTAUNDINV) > 0"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Unidades"] = df_SQL_int["Unidades"].astype(int)

        df_SQL_nits = pd.concat([df_SQL_nits, df_SQL_int], ignore_index= True)

    # df_SQL = df_SQL_nits.groupby("Fecha").sum().reset_index()
    df_SQL_nits
    df_productos = df_SQL_nits.groupby("Codigo producto").sum().reset_index()
    df_productos = df_productos.drop("Unidades", axis= 1)
    df_nits = df_SQL_nits.groupby(["Nitcliente", "Nombre cliente"]).sum().reset_index()
    df_nits = df_nits.drop("Unidades", axis= 1)
    df_nits["Nombre cliente"] = df_nits["Nombre cliente"].str.rstrip()

    return df_SQL_nits, df_productos, df_nits

In [4]:
def nombre_cliente(df, nit):
    a = df[df["Nitcliente"] == nit]["Nombre cliente"].values
    
    return a[0]


In [5]:
df_productos = pd.read_excel("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Databases/Maestra gerencias.xlsx", sheet_name= "Maestra productos BI")
col = ["PRDCOD", "PRDNOM", "Gerente", "Línea CAM"]
df_productos = df_productos[col]
df_productos.columns = ["Codigo producto", "Nombre producto", "Gerencia", "Linea CAM"]

In [6]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

filtro_distrito = 10
filtro_tipozona = "Artes graficas"
filtro_cluster = "B"

df_clusters_f = df_clusters[
                            (df_clusters["Codigo distrito"] == filtro_distrito) &
                            (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                            (df_clusters["Letra cluster"] == filtro_cluster)
                            ]

In [7]:
# Variables auxiliares
lista_nits = []
for nit_suc in df_clusters_f["Nit cliente-sucursal"].tolist():
    lista_nits.append(nit_suc.split("-")[0])

fecha_final = '2023-03-31'
fecha_inicial = '2022-04-01'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')
fecha_inicial = datetime.strptime(fecha_inicial, '%Y-%m-%d').strftime('%Y-%m-%d')

In [8]:
#dfs para EDA y forecast
ventas_cluster, ventas_productos, ventas_nits = df_cluster(nits_clientes= lista_nits, distrito= filtro_distrito, fecha_inicial= fecha_inicial, fecha_final= fecha_final)

In [10]:
productos_cluster = pd.merge(ventas_productos, df_productos, on= "Codigo producto", how= "left")

### Filtro colaborativo basado en usuario:

In [11]:
# Crear una matriz de utilidad de usuarios y películas
utility_matrix = ventas_cluster.pivot(index='Nitcliente', columns='Codigo producto', values='Unidades').fillna(0)
# Calcular la similitud del coseno entre los usuarios
user_similarity = cosine_similarity(utility_matrix)

In [12]:
utility_matrix

Codigo producto,20,40,60,70,290,294,295,296,297,300,...,99703,99705,99706,99708,99709,99710,99716,99723,99724,99726
Nitcliente,,,,,,,,,,,,,,,,,,,,,
19149566,110.0,15.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19322486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19453247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41703985,6.0,7.0,5.0,1.0,0.0,3.0,2.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51941997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901374644,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
901386444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
901409377,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
user_similarity

array([[1.00000000e+00, 1.40865229e-04, 1.80059238e-02, ...,
        2.10171095e-06, 3.35785038e-02, 9.20616353e-03],
       [1.40865229e-04, 1.00000000e+00, 3.95041139e-03, ...,
        5.41798466e-05, 5.10856344e-03, 5.35709319e-03],
       [1.80059238e-02, 3.95041139e-03, 1.00000000e+00, ...,
        1.57173830e-04, 4.11660082e-03, 0.00000000e+00],
       ...,
       [2.10171095e-06, 5.41798466e-05, 1.57173830e-04, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.35785038e-02, 5.10856344e-03, 4.11660082e-03, ...,
        0.00000000e+00, 1.00000000e+00, 5.72652366e-02],
       [9.20616353e-03, 5.35709319e-03, 0.00000000e+00, ...,
        0.00000000e+00, 5.72652366e-02, 1.00000000e+00]])

In [10]:
# Función para obtener las películas recomendadas para un usuario específico
def get_user_recommendations(Nitcliente, num_recommendations=10):
    # Obtener la fila de similitud del usuario
    index = utility_matrix.index.tolist()
    user_similarities = user_similarity[index.index(Nitcliente)]

    # Calcular el promedio ponderado de las clasificaciones de películas de los usuarios similares
    weighted_productos = np.dot(user_similarities, utility_matrix.values) / np.sum(np.abs(user_similarities))

    # Excluir las películas que el usuario ya ha visto
    producto_comprado = utility_matrix.loc[Nitcliente] > 0
    weighted_productos[producto_comprado] = 0

    # Ordenar las películas recomendadas por la clasificación ponderada
    codigos_productos = np.argsort(weighted_productos)[::-1][:num_recommendations]

    # Obtener los títulos de las películas recomendadas
    recommended_productos = productos_cluster.iloc[productos_cluster.index.isin(codigos_productos)]

    return recommended_productos

In [11]:
Nitcliente = 900609253
Nombre_cliente = nombre_cliente(ventas_nits, Nitcliente)
recommended_productos = get_user_recommendations(Nitcliente= Nitcliente, num_recommendations= 10)
print(f"Productos recomendados para el cliente {Nitcliente} {Nombre_cliente}:")
recommended_productos

Productos recomendados para el cliente 900609253 NOVO COMUNICACION GRAFICA:


,Codigo producto,Nombre producto,Gerencia,Linea CAM
0,20,BOND BLANCO,Papel para escritura,Bond
1,40,BOND BLANCO,Papel para escritura,Bond
37,838,REPROGRAF A.B. RESMILLA CARTA,Resmilla,Resmilla - Reprograf
38,839,REPROGRAF A.B. RESMILLA OFICIO,Resmilla,Resmilla - Reprograf
40,1000,PAPEL PERIODICO,Periódico,Periódico
67,5995,FASSON ESM80 SEGURIDAD S1600 SCK 85,Papeles especiales,Autoadhesivos
74,6476,FASSON ART PLUS ESM80/S140/80,Papeles especiales,Autoadhesivos
85,9999,EARTH PACT NATURAL OFFSET,Papel para escritura,Earth Pact
89,10310,BOND BLANCO A.B. PLUS,Papel para escritura,Bond
94,12358,PROPALCOTE ESMALTADO C2S GLOSS,Papel para escritura,Esmaltado


In [12]:
#Añadir explicaciones de las matrices y el trabajo futuro